# Investment Simulation System

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
%pip install pandas_ta_classic

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tqdm 

Note: you may need to restart the kernel to use updated packages.


### Imports

In [4]:
import os
import site

import os
import shutil
import gc
import warnings
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm
from datetime import timedelta

warnings.filterwarnings('ignore')

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

from sklearn.metrics import (
    precision_score, recall_score, f1_score, matthews_corrcoef,
    mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
)


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as Fnn
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [7]:
pandas_ta_classic_path = None
for sp in site.getsitepackages():
    pandas_ta_classic_path = os.path.join(sp, 'pandas_ta_classic')
    if os.path.exists(pandas_ta_classic_path):
        break

if pandas_ta_classic_path:
    squeeze_pro_path = os.path.join(pandas_ta_classic_path, 'momentum', 'squeeze_pro.py')
    if os.path.exists(squeeze_pro_path):
        try:
            with open(squeeze_pro_path, 'r') as f:
                lines = f.readlines()

            new_lines = []
            fixed = False
            for line in lines:
                if "from numpy import NaN as npNaN" in line:
                    new_lines.append(line.replace("from numpy import NaN as npNaN", "# from numpy import NaN as npNaN\nimport numpy as np\n"))
                    fixed = True
                    print("Modified import statement in squeeze_pro.py")
                else:
                    new_lines.append(line)

            if fixed:
                with open(squeeze_pro_path, 'w') as f:
                    f.writelines(new_lines)
                print("Successfully patched pandas_ta_classic/momentum/squeeze_pro.py")
            else:
                print("Could not find the problematic import line in squeeze_pro.py")

        except Exception as e:
            print(f"Error modifying squeeze_pro.py: {e}")
    else:
        print(f"Could not find squeeze_pro.py at {squeeze_pro_path}")
else:
    print("Could not find the pandas_ta_classic library installation path.")

import pandas_ta_classic as ta

Could not find the problematic import line in squeeze_pro.py


### Configurations

In [8]:
MODEL_SAVE_PATH = "trained_models/"
MIN_SEQUENCE_LENGTH = 12  # Minimum sequence length for any company
MAX_SEQUENCE_LENGTH = 12  # Maximum sequence length to cap computational cost
INITIAL_TRAINING_DAYS = 1100  # Number of days to use for initial training only
KELLY_FRACTION = 0.1
SECTOR_CONFIDENCE_THRESHOLD = 0.30
RETRAIN_INTERVAL = 200
MAX_DAY_GAP = 5  # Maximum allowed gap in trading days (to account for weekends/holidays)

In [9]:
MAX_TOTAL_UTILIZATION      = 0.70   # never deploy more than 70% of capital at once
MAX_NEW_DAILY_UTILIZATION  = 0.30   # new trades today ≤ 30% of capital
MAX_TICKER_UTILIZATION     = 0.20   # per ticker cap (sum of all overlapping trades)
MAX_SECTOR_UTILIZATION     = 0.45   # per sector cap (sum across its tickers)
MAX_POSITION_SIZE          = 0.1   # single-trade cap

MAX_POSITIONS_PER_TICKER   = 3      # ladder depth per ticker
MIN_TRADE_DOLLARS          = 100.0  # skip dust trades

KELLY_TRADE_CAP            = 0.30   # cap raw Kelly per trade (before other caps)
DRAWDOWN_THROTTLE_LEVELS   = [(0.05, 0.50), (0.10, 0.25)]  # (peak_dd, utilization_multiplier)


In [10]:
BUCKET_BINS   = [-np.inf, -0.03, -0.01, 0.0, 0.01, 0.03, np.inf]
K = len(BUCKET_BINS) - 1

In [11]:
import random


def set_global_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_global_seeds(42)

In [12]:
CALIBRATION_LOGGED = set()


In [13]:
def bucketize_with_edges(x, edges):
    # x: 1D np array of returns; edges: len K+1
    return np.clip(np.digitize(x, edges[1:-1], right=True), 0, len(edges)-2).astype(np.int64)

def make_cumulative_targets(y_idx, K):
    # y_idx in [0..K-1] -> T: [N,K-1] with T[:,k]=1(y>k)
    y = y_idx.reshape(-1, 1)
    ks = np.arange(K-1).reshape(1, -1)
    return (y > ks).astype(np.float32)

def ordinal_to_class_probs(P_rep):
    # P_rep: [N,K-1] with P(y>k) (after sigmoid), repaired monotone
    N, K1 = P_rep.shape; K = K1+1
    Pc = np.empty((N, K), dtype=np.float32)
    Pc[:,0] = 1.0 - P_rep[:,0]
    for c in range(1, K-1):
        Pc[:,c] = np.clip(P_rep[:,c-1] - P_rep[:,c], 0., 1.)
    Pc[:,K-1] = P_rep[:,K1-1]
    s = Pc.sum(axis=1, keepdims=True)
    return Pc / np.maximum(s, 1e-8)

def monotone_repair_numpy(P):
    P = np.asarray(P, dtype=np.float32).copy()
    for k in range(P.shape[1]-2, -1, -1):
        P[:,k] = np.maximum(P[:,k], P[:,k+1])
    return P

In [14]:


def _temperature_objective(logits: np.ndarray, y: np.ndarray, temperature: float) -> float:
    if temperature <= 0 or not np.isfinite(temperature):
        return np.inf
    P_cum = 1.0 / (1.0 + np.exp(-logits / temperature))
    P_cum = monotone_repair_numpy(P_cum)
    Pc = ordinal_to_class_probs(P_cum)
    idx = np.arange(len(y))
    eps = 1e-9
    return -np.log(np.clip(Pc[idx, y], eps, 1.0)).mean()


# Multiclass calibration tuned for macro-F1 balance
def _class_dist(y_hat, K):
    hist = np.bincount(y_hat, minlength=K).astype(np.float32)
    return hist / max(1, y_hat.size)

def _balance_penalty(p_hat, p_target, norm="l1"):
    if norm == "l1":
        return float(np.abs(p_hat - p_target).sum())
    elif norm == "l2":
        return float(np.sqrt(((p_hat - p_target) ** 2).sum()))
    else:
        return float(np.abs(p_hat - p_target).sum())

def decode_ordinal_with_taus(P_rep, taus=None, taus_override=None):
    # Decode ordinal cumulative probs to class indices using per-head taus.
    N, K_1 = P_rep.shape
    if taus is None:
        taus = np.full(K_1, 0.5, dtype=np.float32)
    if taus_override:
        taus = taus.copy()
        for k, v in taus_override.items():
            taus[k] = v
    comp = (P_rep >= taus.reshape(1, -1)).astype(np.int32)
    return comp.sum(axis=1).astype(np.int64)

def tune_temperature_and_taus(
    Z_val,
    y_val_idx,
    K,
    T_grid=np.linspace(0.6, 2.5, 15),
    tau_grid=np.linspace(0.2, 0.8, 31),
    lambda_balance=0.05,
    target_priors="uniform",
    coord_rounds=3,
):
    """Learn a temperature T and per-head taus by maximizing macro-F1 with a balance penalty."""
    if isinstance(Z_val, torch.Tensor):
        Z_val = Z_val.detach().cpu().numpy()

    N, K_1 = Z_val.shape
    assert K_1 == K - 1

    if isinstance(target_priors, str) and target_priors == "uniform":
        p_target = np.ones(K, dtype=np.float32) / K
    else:
        p_target = np.asarray(target_priors, dtype=np.float32)
        p_target = p_target / p_target.sum()

    best = {"score": -1e9, "T": 1.0, "taus": np.full(K_1, 0.5, np.float32)}

    for T in T_grid:
        P_cum = 1.0 / (1.0 + np.exp(-(Z_val / T)))
        P_rep = monotone_repair_numpy(P_cum)

        taus = np.full(K_1, 0.5, dtype=np.float32)
        for _ in range(coord_rounds):
            for k in range(K_1):
                best_k = (taus[k], -1e9)
                for tau in tau_grid:
                    taus_try = taus.copy()
                    taus_try[k] = tau
                    y_hat = decode_ordinal_with_taus(P_rep, taus_try)
                    f1 = f1_score(y_val_idx, y_hat, average='macro', zero_division=0)
                    p_hat = _class_dist(y_hat, K)
                    score = f1 - lambda_balance * _balance_penalty(p_hat, p_target, norm="l1")
                    if score > best_k[1]:
                        best_k = (tau, score)
                taus[k] = best_k[0]

        y_hat = decode_ordinal_with_taus(P_rep, taus)
        f1 = f1_score(y_val_idx, y_hat, average='macro', zero_division=0)
        p_hat = _class_dist(y_hat, K)
        score = f1 - lambda_balance * _balance_penalty(p_hat, p_target, norm="l1")

        if score > best["score"]:
            best.update({"score": score, "T": float(T), "taus": taus.astype(np.float32)})

    return best["T"], best["taus"]


#### Define Horizon Target
For a horizon H, compute both the direction and H-day return off the same base day t

In [15]:
def add_horizon_targets(df: pd.DataFrame, H: int, price_col='close_price') -> pd.DataFrame:
    df = df.sort_values(['ticker','date']).copy()
    df[f'ret_{H}d']    = df.groupby('ticker')[price_col].shift(-H) / df[price_col] - 1.0
    df[f'target_{H}d'] = (df[f'ret_{H}d'] > 0).astype(int)
    return df


#### Identify Contiguous Periods
Groups rows for a ticker into blocks where data gaps never exceed MAX_DAY_GAP

In [16]:
def identify_contiguous_periods(df: pd.DataFrame, max_gap_days: int = MAX_DAY_GAP) -> list:

    if df.empty:
        return []

    df = df.sort_values('date').reset_index(drop=True)
    dates = pd.to_datetime(df['date'])

    contiguous_periods = []
    start_idx = 0

    for i in range(1, len(dates)):
        gap = (dates[i] - dates[i-1]).days
        if gap > max_gap_days:
            # End current period and start new one
            contiguous_periods.append((start_idx, i-1))
            start_idx = i


    contiguous_periods.append((start_idx, len(dates)-1))


    contiguous_periods = [(s, e) for s, e in contiguous_periods if e - s >= MIN_SEQUENCE_LENGTH]

    return contiguous_periods

#### Calculate Dynamic Sequence Length 
Pick a sequence length tailored to the amount of history available for a ticker.
Avoids hard-coding a single window size for short vs long histories.

In [17]:
def calculate_dynamic_sequence_length(company_data_df: pd.DataFrame,
                                     min_length: int = MIN_SEQUENCE_LENGTH,
                                     max_length: int = MAX_SEQUENCE_LENGTH,
                                     target_fraction: float = 0.15) -> int:

    total_days = len(company_data_df)


    dynamic_length = int(total_days * target_fraction)
    dynamic_length = max(min_length, min(dynamic_length, max_length))

    return dynamic_length

#### Create Contiguous Sequences
Turn each contiguous period into overlapping (sequence_length) windows (X) and next-step labels (y).
Produces the actual training/validation tensors for the model.

In [18]:
def create_contiguous_sequences(data: np.ndarray, targets: np.ndarray,
                               contiguous_periods: list, sequence_length: int, H: int):

    X, y = [], []

    for start_idx, end_idx in contiguous_periods:
        usable_end = end_idx - H
        period_length = usable_end - start_idx + 1
        if period_length < sequence_length + 1:
            continue

        for i in range(start_idx + sequence_length -1 , usable_end + 1):
            X.append(data[i-sequence_length + 1 : i +1])
            y.append(targets[i + H])

    return np.array(X) if X else np.array([]), np.array(y) if y else np.array([])

#### Create Target Variable
Build the binary classification target per row.

In [19]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:

    print("Creating target variable...")
    df = df.sort_values(by=['ticker', 'date']).copy()
    df['next_day_close'] = df.groupby('ticker')['close_price'].shift(-1)
    df['target'] = (df['next_day_close'] > df['close_price']).astype(int)
    df.dropna(subset=['next_day_close'], inplace=True)
    df['target'] = df['target'].astype(int)
    print("Target variable created.")
    return df

#### Track Current Exposures from Open Trades
Settle exits first each day, then compute exposure books before opening new ones.

In [20]:
from collections import defaultdict

def compute_exposures(open_positions, sectors_by_ticker):
    expo_ticker = defaultdict(float)
    expo_sector = defaultdict(float)
    total = 0.0
    for pos in open_positions:
        amt = pos['invest']
        tkr = pos['ticker']
        sec = sectors_by_ticker.get(tkr, 'UNKNOWN')
        expo_ticker[tkr] += amt
        expo_sector[sec] += amt
        total += amt
    return total, dict(expo_ticker), dict(expo_sector)

#### Throttle Book When in Drawdown 
Apply multiplicative haircut to utilisation caps based on current peak to trough drawdown 

In [21]:
def utilization_throttle(equity_curve):
    if not equity_curve:
        return 1.0
    peak = max(equity_curve)
    curr = equity_curve[-1]
    dd = 0.0 if peak == 0 else (peak - curr)/peak
    mult = 1.0
    for level, m in DRAWDOWN_THROTTLE_LEVELS:
        if dd >= level:
            mult = min(mult, m)
    return mult

#### Cap Each Proposed Trade
When sizing a new trade, clamp it by all remaining budgets

In [22]:
def clamp_by_caps(proposed_amt, capital, 
                  total_open, new_today_open, expo_ticker, expo_sector,
                  ticker, sector,
                  util_mult=1.0):
    # remaining dollar budgets
    rem_total  = MAX_TOTAL_UTILIZATION*util_mult*capital - total_open
    rem_new    = MAX_NEW_DAILY_UTILIZATION*util_mult*capital - new_today_open
    rem_ticker = MAX_TICKER_UTILIZATION*util_mult*capital - expo_ticker.get(ticker, 0.0)
    rem_sector = MAX_SECTOR_UTILIZATION*util_mult*capital - expo_sector.get(sector, 0.0)
    per_trade  = MAX_POSITION_SIZE*util_mult*capital

    # clamp
    max_affordable = max(0.0, min(proposed_amt, rem_total, rem_new, rem_ticker, rem_sector, per_trade))
    return max_affordable

#### Calculate Historical Payouts
Estimate average upside “b” per winning trade for Kelly sizing. 

Only looks at wins; losses are implicit in Kelly’s (1-p)/b term.


In [23]:

def calculate_historical_payouts(df, ret_col: str = None):
    """
    Returns a dict per ticker with:
      - 'b': avg_win / avg_loss_abs  (Kelly 'odds' ratio)
      - 'avg_win': mean positive return for the chosen horizon
      - 'avg_loss_abs': mean absolute negative return for the chosen horizon
    Uses the same horizon return column as the multiclass model (ret_{H}d).
    """
    if ret_col is None:
        ret_cols = [c for c in df.columns if c.startswith('ret_') and c.endswith('d')]
        ret_col = ret_cols[0] if ret_cols else None
    if ret_col is None or ret_col not in df.columns:
        raise ValueError("Return column for payout computation not found.")

    out = {}
    for t, g in df.groupby('ticker'):
        r = g[ret_col]
        wins = r[r > 0]
        losses = r[r < 0]
        avg_win = wins.mean() if not wins.empty else np.nan
        avg_loss_abs = (-losses).mean() if not losses.empty else np.nan

        if np.isfinite(avg_win) and np.isfinite(avg_loss_abs) and avg_loss_abs > 0:
            b = avg_win / avg_loss_abs
        else:
            b = np.nan

        out[t] = {'b': b, 'avg_win': avg_win, 'avg_loss_abs': avg_loss_abs}
    return out


##### PyTorch Helpers

In [24]:
class OrdinalSequenceDataset(Dataset):
    def __init__(self, X, T):  # T: float32 matrix [N, K-1] with 0/1
        self.X = torch.tensor(X, dtype=torch.float32)
        self.T = torch.tensor(T, dtype=torch.float32)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.T[i]
    
class LSTMOrdinal(nn.Module):
    def __init__(self, n_features, hidden1=128, hidden2=64, fc=32, dropout=0.3, inter_dropout=0.1, head='CORAL', K=6):
        super().__init__()
        self.K = K
        self.lstm1 = nn.LSTM(n_features, hidden1, num_layers=1, batch_first=True, dropout=0.1)
        self.inter_drop = nn.Dropout(inter_dropout)
        self.lstm2 = nn.LSTM(hidden1, hidden2, num_layers=1, batch_first=True, dropout=0.1)

        self.norm = nn.BatchNorm1d(hidden2)
        self.fc1 = nn.Linear(hidden2, fc)
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(dropout)

        if head.upper() == 'CORAL':
            self.w = nn.Linear(fc, 1, bias=False)
            self._beta = nn.Parameter(torch.zeros(K-1))
            self.softplus = nn.Softplus()
            self.head_type = 'CORAL'
        else:
            self.fc_out = nn.Linear(fc, K-1)
            self.head_type = 'CORN'

    def forward(self, x):
        out1,_ = self.lstm1(x)
        out1 = self.inter_drop(out1)
        out2,_ = self.lstm2(out1)
        last = out2[:, -1, :]
        last = self.norm(last)
        z = self.relu(self.fc1(last))
        z = self.drop(z)
        if self.head_type == 'CORAL':
            base = self.w(z)                              # [B,1]
            deltas = self.softplus(self._beta)            # ≥0
            b = torch.cumsum(deltas, dim=0)               # [K-1]
            logits = base - b                              # broadcast -> [B,K-1]
        else:
            logits = self.fc_out(z)                        # [B,K-1]
        return logits

class EarlyStopper:
    def __init__(self, patience=15, mode='min'):
        self.patience = patience
        self.counter = 0
        self.best_metric = None
        self.best_state_dict = None
        self.mode = mode  # 'min' for val_loss
    def step(self, metric, model):
        if self.best_metric is None:
            improved = True
        else:
            improved = (metric < self.best_metric) if self.mode == 'min' else (metric > self.best_metric)
        if improved:
            self.best_metric = metric
            self.best_state_dict = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            self.counter = 0
        else:
            self.counter += 1
        return improved

@torch.no_grad()
def _evaluate(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    all_logits = []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        total_loss += loss.item() * xb.size(0)
        all_logits.append(logits.detach().cpu())
    avg_loss = total_loss / len(loader.dataset)
    logits = torch.cat(all_logits, dim=0).numpy()
    probs = 1.0 / (1.0 + np.exp(-logits))  # sigmoid
    return avg_loss, probs

#### Train Company Models
Train, calibrate, and persist a per-ticker classifier.
This encapsulates per-asset modelling with proper scaling and probability calibration, which is crucial since Kelly needs probabilities (not raw logits).

In [25]:

def train_company_models(company_data_df: pd.DataFrame,
                                 ticker: str,
                                 feature_cols: list,
                                 model_save_path: str,
                                 sequence_length: int = None,
                                 H: int = 1,
                                 K: int = K,        # global K
                                 head_type: str = 'CORAL'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f"[TRAIN] {ticker}: preparing data (H={H}) with {len(company_data_df)} rows")

    ret_col = f"ret_{H}d"
    if ret_col not in company_data_df.columns:
        company_data_df = add_horizon_targets(company_data_df, H=H, price_col='close_price')

    if sequence_length is None:
        sequence_length = calculate_dynamic_sequence_length(company_data_df)
    if len(company_data_df) < sequence_length + H + 10:
        print(f"[TRAIN] {ticker}: insufficient history for sequence length {sequence_length}")
        return False, sequence_length

    contigs = identify_contiguous_periods(company_data_df)
    if not contigs:
        print(f"[TRAIN] {ticker}: no contiguous periods found")
        return False, sequence_length

    rets = company_data_df[ret_col].values
    y_idx_all = bucketize_with_edges(rets, BUCKET_BINS)
    X_all = company_data_df[feature_cols].values

    X_seq, y_seq = create_contiguous_sequences(
        X_all, y_idx_all, contigs, sequence_length, H
    )
    if len(X_seq) < 40:
        print(f"[TRAIN] {ticker}: too few sequences ({len(X_seq)})")
        return False, sequence_length
    print(f"[TRAIN] {ticker}: built {len(X_seq)} sequences (seq_len={sequence_length})")

    X_tr, X_val, y_tr, y_val = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

    scaler = MinMaxScaler()
    F = X_tr.shape[-1]
    X_tr = scaler.fit_transform(X_tr.reshape(-1, F)).reshape(X_tr.shape)
    X_val = scaler.transform(X_val.reshape(-1, F)).reshape(X_val.shape)
    print(f"[TRAIN] {ticker}: scaler fitted on {X_tr.shape[0]} training sequences")

    T_tr  = make_cumulative_targets(y_tr, K)
    T_val = make_cumulative_targets(y_val, K)

    train_ds = OrdinalSequenceDataset(X_tr, T_tr)
    val_ds   = OrdinalSequenceDataset(X_val, T_val)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
    val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, drop_last=False)

    n_features = X_tr.shape[-1]
    model = LSTMOrdinal(n_features=n_features, head=head_type, K=K).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

    pos_rate = T_tr.mean(axis=0)
    pos_w = ((1.0 - pos_rate) / np.clip(pos_rate, 1e-6, 1.0)).astype(np.float32)
    pos_w = torch.tensor(pos_w, dtype=torch.float32, device=device)

    def loss_fn(logits, T):
        losses = []
        for k in range(logits.shape[1]):
            losses.append(Fnn.binary_cross_entropy_with_logits(logits[:,k], T[:,k], pos_weight=pos_w[k]))
        return torch.stack(losses).mean()

    early = EarlyStopper(patience=15, mode='min')
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=7, min_lr=1e-7)

    for epoch in range(100):
        model.train()
        tot = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad(set_to_none=True)
            logits = model(xb)
            l = loss_fn(logits, yb)
            l.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            tot += l.item() * xb.size(0)

        model.eval()
        with torch.no_grad():
            vtot = 0.0
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                l = loss_fn(logits, yb)
                vtot += l.item() * xb.size(0)
            val_loss = vtot / len(val_loader.dataset)

        scheduler.step(val_loss)
        _ = early.step(val_loss, model)
        if early.counter >= early.patience:
            break

    if early.best_state_dict is not None:
        model.load_state_dict(early.best_state_dict)

    ret_seq_full = company_data_df[ret_col].values[sequence_length:]
    ret_train = ret_seq_full[:len(y_tr)]
    mu_c = np.array([ret_train[y_tr==c].mean() if np.any(y_tr==c) else 0.0 for c in range(K)], dtype=np.float32)


    taus = None
    temperature = 1.0
    try:
        model.eval()
        batch = 256
        logits_list = []
        for start in range(0, len(X_val), batch):
            xb = torch.from_numpy(X_val[start:start+batch].astype(np.float32)).to(device)
            with torch.no_grad():
                logits = model(xb)
            logits_list.append(logits.cpu().numpy())
        if logits_list:
            val_logits = np.vstack(logits_list)
            T_star, taus = tune_temperature_and_taus(
                Z_val=val_logits,
                y_val_idx=y_val,
                K=K,
                T_grid=np.linspace(0.6, 2.5, 15),
                tau_grid=np.linspace(0.2, 0.8, 31),
                lambda_balance=0.05,
                target_priors="uniform",
                coord_rounds=3,
            )
            temperature = float(T_star)
            print(f"[CAL] {ticker}: taus={np.round(taus, 3)} | temperature={temperature:.3f}")
        else:
            print(f"[CAL] {ticker}: no validation logits -> skipping calibration")
    except Exception as exc:
        print(f"[CAL] {ticker}: calibration failed ({exc})")
        taus = None
        temperature = 1.0

    os.makedirs(model_save_path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(model_save_path, f"{ticker}_lstmORD_H{H}.pt"))
    joblib.dump(scaler, os.path.join(model_save_path, f"{ticker}_scaler_H{H}.pkl"))
    joblib.dump(sequence_length, os.path.join(model_save_path, f"{ticker}_seq_length_H{H}.pkl"))
    joblib.dump(BUCKET_BINS, os.path.join(model_save_path, f"{ticker}_edges_H{H}.pkl"))
    joblib.dump(mu_c, os.path.join(model_save_path, f"{ticker}_mu_c_H{H}.pkl"))
    if taus is not None and len(taus) == (K - 1) and np.all(np.isfinite(taus)):
        joblib.dump(taus, os.path.join(model_save_path, f"{ticker}_taus_H{H}.pkl"))
        print(f"[SAVE] {ticker}: stored taus")
    if temperature is not None and np.isfinite(temperature):
        joblib.dump(float(temperature), os.path.join(model_save_path, f"{ticker}_temperature_H{H}.pkl"))
        print(f"[SAVE] {ticker}: stored temperature scale {temperature:.3f}")
    print(f"[SAVE] {ticker}: artifacts persisted (H={H})")

    del model, scaler, train_loader, val_loader, train_ds, val_ds
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    return True, sequence_length



#### Predict Next Day Performance 
One-step-ahead inference for a ticker on a given day.

Feeds calibrated p(up) into the portfolio/Kelly step.

In [26]:

def predict_next_horizon(company_data_df: pd.DataFrame,
                         ticker: str,
                         feature_cols: list,
                         model_save_path: str,
                         H: int,
                         K: int = 6,
                         head: str = "CORAL") -> dict | None:
    """
    Ordinal inference with optional calibration artifacts. Returns class probs plus
    summary stats so the simulator can log decisions.
    """
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        state_dict_path = os.path.join(model_save_path, f"{ticker}_lstmORD_H{H}.pt")
        if not os.path.isfile(state_dict_path):
            return None

        scaler   = joblib.load(os.path.join(model_save_path, f"{ticker}_scaler_H{H}.pkl"))
        seq_len  = joblib.load(os.path.join(model_save_path, f"{ticker}_seq_length_H{H}.pkl"))
        mu_c     = joblib.load(os.path.join(model_save_path, f"{ticker}_mu_c_H{H}.pkl"))

        taus_path = os.path.join(model_save_path, f"{ticker}_taus_H{H}.pkl")
        T_path    = os.path.join(model_save_path, f"{ticker}_temperature_H{H}.pkl")
        taus = joblib.load(taus_path) if os.path.isfile(taus_path) else None
        temperature = float(joblib.load(T_path)) if os.path.isfile(T_path) else 1.0

        n_features = len(feature_cols)
        model = LSTMOrdinal(n_features=n_features, head=head.upper(), K=K).to(device)
        model.load_state_dict(torch.load(state_dict_path, map_location=device))
        model.eval()
    except Exception:
        return None

    if 'CALIBRATION_LOGGED' not in globals():
        globals()['CALIBRATION_LOGGED'] = set()
    if ticker not in CALIBRATION_LOGGED:
        if taus is not None:
            print(f"[CAL] {ticker}: loaded taus {np.round(taus,3)}")
        else:
            print(f"[CAL] {ticker}: no taus artifact found")
        print(f"[CAL] {ticker}: temperature scaling {temperature:.3f}")
        CALIBRATION_LOGGED.add(ticker)

    contiguous = identify_contiguous_periods(company_data_df)
    if not contiguous:
        return None
    start, end = contiguous[-1]
    period = company_data_df.iloc[start:end+1]
    if len(period) < seq_len:
        return None

    try:
        last_seq = period.tail(seq_len)
        X_scaled = scaler.transform(last_seq[feature_cols])
    except Exception:
        return None

    x = torch.from_numpy(np.asarray([X_scaled], dtype=np.float32)).to(device)

    with torch.no_grad():
        z = model(x)
        if temperature and temperature > 0 and temperature != 1.0:
            z = z / temperature
        P_cum = torch.sigmoid(z).cpu().numpy()
        P_cum = monotone_repair_numpy(P_cum)
        Pc    = ordinal_to_class_probs(P_cum)

    mu_c = np.asarray(mu_c, dtype=np.float32).reshape(-1)
    if mu_c.shape[0] != K:
        return None
    expected_rise = float((Pc[0] * mu_c).sum())

    top_bucket_idx = K - 1
    prob_top_bucket = float(Pc[0, top_bucket_idx])
    bucket_idx = int(np.argmax(Pc[0]))

    class_idx = None
    if taus is not None:
        taus = np.asarray(taus, dtype=np.float32).reshape(-1)
        if taus.shape[0] == (K - 1):
            class_idx = int(decode_ordinal_with_taus(P_cum, taus=taus)[0])

    positive_buckets = [i for i in range(K) if BUCKET_BINS[i] >= 0]
    prob_up = float(Pc[0, positive_buckets].sum()) if positive_buckets else prob_top_bucket

    return {
        'ticker': ticker,
        'expected_rise': expected_rise,
        'predicted_prob': prob_up,  # probability of positive move
        'prob_up': prob_up,
        'prob_bucket_top': prob_top_bucket,
        'bucket_idx': bucket_idx,
        'class_idx': class_idx,
        'class_probs': Pc[0].tolist(),
    }


#### Select and Size Portfolio
Turn a cross-section of calibrated predictions into position sizes.

Enforces diversification (one per sector) and risk discipline (Fractional Kelly).

In [27]:

def select_and_size_portfolio(daily_predictions_df: pd.DataFrame, payout_map: dict,
                              total_capital: float, sector_threshold: float,
                              kelly_fraction: float) -> pd.DataFrame:
    """Pick strongest candidates per sector and size using Kelly on prob_up and horizon payoffs."""
    if daily_predictions_df.empty:
        print("[PORT] No predictions provided today.")
        return pd.DataFrame()

    print(f"[PORT] Received {len(daily_predictions_df)} predictions for sizing.")
    df = daily_predictions_df.copy()

    if 'prob_up' not in df.columns and 'class_probs' in df.columns:
        positive_buckets = [i for i in range(K) if BUCKET_BINS[i] >= 0]
        def _sum_up(probs):
            if isinstance(probs, (list, tuple)) and len(probs) == K and positive_buckets:
                return float(sum(probs[i] for i in positive_buckets))
            return np.nan
        df['prob_up'] = df['class_probs'].apply(_sum_up)

    if 'prob_up' not in df.columns and 'predicted_prob' in df.columns:
        df['prob_up'] = df['predicted_prob']

    if 'expected_rise' not in df.columns:
        print("[PORT] Missing expected_rise; cannot size portfolio.")
        return pd.DataFrame()

    df = df[(df['prob_up'] >= sector_threshold) & (df['expected_rise'] > 0)]
    print(f"[PORT] {len(df)} tickers cleared prob_up ≥ {sector_threshold:.3f} and positive expected return.")
    if df.empty:
        return pd.DataFrame()

    winners = (df.sort_values(['sector', 'expected_rise'], ascending=[True, False])
                 .groupby('sector', as_index=False)
                 .head(1))
    if winners.empty:
        print("[PORT] No sector winners after grouping.")
        return pd.DataFrame()

    rows = []
    for _, row in winners.iterrows():
        ticker = row['ticker']
        sector = row['sector']
        p_up = float(row['prob_up'])
        payout = payout_map.get(ticker, {})
        b = payout.get('b') if isinstance(payout, dict) else float(payout)
        if not np.isfinite(b) or b <= 0:
            print(f"[PORT] Sector {sector}: skipping {ticker} due to invalid payout ratio {b}.")
            continue
        kelly_pct = p_up - (1.0 - p_up) / b
        if kelly_pct <= 0:
            print(f"[PORT] Sector {sector}: Kelly <= 0 for {ticker} (p_up={p_up:.3f}, b={b:.3f}).")
            continue
        investment_fraction = min(MAX_POSITION_SIZE, kelly_pct * kelly_fraction)
        investment_fraction = max(0.0, investment_fraction)
        investment_amount = total_capital * investment_fraction
        if investment_amount < MIN_TRADE_DOLLARS:
            print(f"[PORT] Sector {sector}: allocation ${investment_amount:.2f} below minimum; skipping {ticker}.")
            continue
        print(f"[PORT] Sector {sector}: selecting {ticker} (p_up={p_up:.3f}, kelly={kelly_pct:.3f}) "
              f"with ${investment_amount:.2f} allocation")
        rows.append({
            'ticker': ticker,
            'sector': sector,
            'prob_up': p_up,
            'predicted_prob': p_up,
            'expected_rise': row['expected_rise'],
            'bucket_idx': row.get('bucket_idx', np.nan),
            'investment_fraction': investment_fraction,
            'investment_amount': investment_amount
        })

    if not rows:
        print("[PORT] All sector winners rejected after Kelly sizing.")
        return pd.DataFrame()

    return pd.DataFrame(rows)


#### Models Exist for Ticker
Quick guard to avoid retraining if a ticker’s artifacts already exist.

Decide whether to train.

In [28]:
def models_exist_for_ticker(ticker: str, model_path: str, H: int = 1) -> bool:
    """Check the minimal set of artifacts needed for ordinal inference."""
    required = [
        f"{ticker}_lstmORD_H{H}.pt",      # model weights (ordinal head)
        f"{ticker}_scaler_H{H}.pkl",      # feature scaler
        f"{ticker}_seq_length_H{H}.pkl",  # sequence length
        f"{ticker}_mu_c_H{H}.pkl",        # per-class train means μ_c for expected return
    ]
    return all(os.path.exists(os.path.join(model_path, f)) for f in required)


#### Run Simulation
The “engine” — walks forward over dates, trains as needed, predicts, sizes, books PnL, and updates capital.

Enforces chronological integrity, periodic retraining, diversification, and proper capital tracking.

In [29]:
def run_simulation(master_df: pd.DataFrame, payout_map: dict, feature_cols: list,
                   initial_capital: float, initial_training_days: int = INITIAL_TRAINING_DAYS,
                   H: int = 1, allow_overlap: bool = True):

    print(f"\nStarting simulation with {initial_training_days} initial training days... (H={H})")
    capital = initial_capital
    equity_curve = [capital]
    simulation_log = []

    # Precompute horizon returns if not already present
    ret_col = f"ret_{H}d"
    tgt_col = f"target_{H}d"
    if ret_col not in master_df.columns or tgt_col not in master_df.columns:
        print("Adding horizon targets to master_df...")
        master_df = add_horizon_targets(master_df, H=H, price_col='close_price')

    sectors_by_ticker = master_df.groupby('ticker')['sector'].first().to_dict()

    all_tickers = master_df['ticker'].unique()
    retrain_counter = {ticker: 0 for ticker in all_tickers}
    ticker_sequence_lengths = {}

    unique_dates = sorted(master_df['date'].unique())
    start_index = min(initial_training_days, len(unique_dates) - 1)
    print(f"Starting predictions from day {start_index} (after {initial_training_days} training days)")

    # Overlapping book
    open_positions = []  # each: {'ticker','invest','entry_date','exit_date'}
    prev_equity_end = capital

    for i in tqdm(range(start_index, len(unique_dates)), desc="Simulating Trading Days"):
        current_date = unique_dates[i]

        # settle positions exiting today
        total_pnl_today = 0.0
        still_open = []
        for pos in open_positions:
            if pos['exit_date'] == current_date:
                # realize H-day return measured from entry date
                row = master_df[(master_df['date'] == pos['entry_date']) & (master_df['ticker'] == pos['ticker'])]
                if not row.empty:
                    rH = row.iloc[0][ret_col]
                    if pd.notna(rH):
                        pnl = pos['invest'] * rH
                        total_pnl_today += pnl
                        print(f"[EXIT] {current_date}: {pos['ticker']} r={rH:.4f}, PnL=${pnl:.2f}")
                        capital += pos['invest'] + pnl  # release capital
                # else: silently drop if missing
            else:
                still_open.append(pos)
        open_positions = still_open

        # equity at START (after settlements, before new entries)
        open_notional_start = sum(p['invest'] for p in open_positions)
        equity_start = capital + open_notional_start
        daily_return = (equity_start / prev_equity_end) - 1.0 if prev_equity_end > 0 else 0.0

        # build prediction universe using info strictly before current_date
        historical_data = master_df[master_df['date'] < current_date]
        if i == 0:  # no "yesterday" slice for display
            todays_data_for_prediction = pd.DataFrame(columns=master_df.columns)
        else:
            todays_data_for_prediction = master_df[master_df['date'] == unique_dates[i-1]]

        daily_predictions = []
        for ticker in todays_data_for_prediction['ticker'].unique():
            company_hist_data = historical_data[historical_data['ticker'] == ticker]
            if company_hist_data.empty:
                continue

            need_retrain = retrain_counter.get(ticker, 0) >= RETRAIN_INTERVAL
            have_models  = models_exist_for_ticker(ticker, MODEL_SAVE_PATH, H=H)
            if need_retrain or not have_models:
                print("Retraining model for ticker:", ticker)
                training_success, seq_length = train_company_models(
                    company_hist_data, ticker, feature_cols, MODEL_SAVE_PATH, H=H
                )
                if training_success:
                    ticker_sequence_lengths[ticker] = seq_length
                    retrain_counter[ticker] = 0
                else:
                    continue

            pred = predict_next_horizon(company_hist_data, ticker, feature_cols, MODEL_SAVE_PATH, H=H)
            if pred:
                info = todays_data_for_prediction[todays_data_for_prediction['ticker'] == ticker].iloc[0]
                pred.update({'company_name': info['company_name'], 'sector': info['sector']})
                daily_predictions.append(pred)
                retrain_counter[ticker] += 1

        daily_predictions_df = pd.DataFrame(daily_predictions)
        print(f"[SIM] {current_date}: built {len(daily_predictions_df)} predictions")
        if (not daily_predictions_df.empty and
            'predicted_prob' not in daily_predictions_df.columns and
            'calibrated_prediction' not in daily_predictions_df.columns):
            print("[WARN] predicted_prob missing from daily_predictions_df columns:",
                  sorted(daily_predictions_df.columns))


        # size raw trades via Kelly
        investment_decision_df = select_and_size_portfolio(
            daily_predictions_df,
            payout_map,
            capital,
            sector_threshold=SECTOR_CONFIDENCE_THRESHOLD,
            kelly_fraction=KELLY_FRACTION,
        )
        if investment_decision_df.empty:
            print(f"[SIM] {current_date}: no trades proposed today.")
        else:
            print(f"[SIM] {current_date}: {len(investment_decision_df)} trades proposed.")

        # entry logic (allow_overlap controls laddering)
        exit_idx = i + H
        if exit_idx < len(unique_dates):
            exit_date = unique_dates[exit_idx]
        else:
            exit_date = None  # can't open new trades that we cannot exit within data

        entries_today = []
        if exit_date is not None and not investment_decision_df.empty:
            # ensure the column exists even if future changes rename it
            if 'predicted_prob' not in investment_decision_df.columns:
                if 'calibrated_prediction' in investment_decision_df.columns:
                    investment_decision_df = investment_decision_df.rename(
                        columns={'calibrated_prediction': 'predicted_prob'}
                    )
                elif 'score' in investment_decision_df.columns:
                    investment_decision_df = investment_decision_df.rename(
                        columns={'score': 'predicted_prob'}
                    )
                else:
                    # nothing to sort by
                    investment_decision_df = pd.DataFrame()

            if not investment_decision_df.empty:
                investment_decision_df = investment_decision_df.sort_values('predicted_prob', ascending=False)


            # current exposures (post-settlement)
            util_mult = utilization_throttle(equity_curve)
            total_open, expo_ticker, expo_sector = compute_exposures(open_positions, sectors_by_ticker)
            new_today_open = 0.0

            for _, trade in investment_decision_df.iterrows():
                tkr = trade['ticker']
                sec = sectors_by_ticker.get(tkr, 'UNKNOWN')

                # per-ticker ladder depth
                open_count = sum(1 for p in open_positions if p['ticker'] == tkr)
                if allow_overlap:
                    if open_count >= MAX_POSITIONS_PER_TICKER:
                        continue
                else:
                    if open_count >= 1:
                        continue  # no overlapping allowed

                # base size from Kelly (already scaled by KELLY_FRACTION)
                base_amt = float(trade['investment_amount'])
                base_amt = min(base_amt, KELLY_TRADE_CAP * (capital + sum(p['invest'] for p in open_positions)))  # vs equity

                # clamp by remaining budgets
                amt = clamp_by_caps(base_amt, (capital + sum(p['invest'] for p in open_positions)),
                                    total_open, new_today_open, expo_ticker, expo_sector,
                                    tkr, sec, util_mult)
                if amt < MIN_TRADE_DOLLARS:
                    continue

                # lock capital and record position
                capital -= amt
                total_open += amt
                new_today_open += amt
                expo_ticker[tkr] = expo_ticker.get(tkr, 0.0) + amt
                expo_sector[sec] = expo_sector.get(sec, 0.0) + amt

                pos = {'ticker': tkr, 'invest': amt, 'entry_date': current_date, 'exit_date': exit_date}
                open_positions.append(pos)
                entries_today.append({**trade.to_dict(), 'allocated_amount': amt, 'exit_date': exit_date})
                print(f"[TRADE] {current_date}: allocated ${amt:.2f} to {tkr} exiting on {exit_date}")

        # equity at END (after entries)
        open_notional_end = sum(p['invest'] for p in open_positions)
        equity_end = capital + open_notional_end

        # log the day
        simulation_log.append({
            'date': current_date,
            'capital_start': equity_start - open_notional_start,  # cash only
            'capital_end': capital,                               # cash only
            'equity_start': equity_start,
            'equity_end': equity_end,
            'daily_pnl_realized': total_pnl_today,                # realized PnL today
            'daily_return': daily_return,
            'investments_made': entries_today
        })

        equity_curve.append(equity_end)
        prev_equity_end = equity_end

    return pd.DataFrame(simulation_log)


#### Calculate Final Results


In [30]:
def calculate_final_results(simulation_log: pd.DataFrame, initial_capital: float, rf_annual: float = 0.0):
    if simulation_log.empty:
        print("Simulation log is empty. No results to calculate.")
        return

    use_equity = {'equity_start', 'equity_end'}.issubset(simulation_log.columns)

    # precomputed daily_return if available and sane
    if 'daily_return' in simulation_log.columns:
        daily_returns = simulation_log['daily_return'].astype(float).values
    elif use_equity:
        eq_start = simulation_log['equity_start'].astype(float).values
        eq_end   = simulation_log['equity_end'].astype(float).values
        # r_t = equity_start_t / equity_end_{t-1} - 1
        prev_end = np.roll(eq_end, 1)
        prev_end[0] = eq_end[0] if eq_end[0] != 0 else eq_start[0]
        daily_returns = (eq_start / prev_end) - 1.0
    else:
        # cash-based (not ideal for H>0)
        print("Warning: Using cash-based returns")
        cap_start = simulation_log['capital_start'].astype(float).values
        cap_end   = simulation_log['capital_end'].astype(float).values
        prev_end  = np.roll(cap_end, 1)
        prev_end[0] = cap_end[0] if cap_end[0] != 0 else cap_start[0]
        daily_returns = (cap_start / prev_end) - 1.0

    daily_returns = np.where(np.isfinite(daily_returns), daily_returns, 0.0)

    # ROI from final equity if present, else cash
    end_col = 'equity_end' if use_equity else 'capital_end'
    final_value = float(simulation_log[end_col].iloc[-1])
    total_roi = (final_value / initial_capital) - 1.0

    # Sharpe
    excess = daily_returns - (rf_annual / 252.0)
    vol = excess.std(ddof=1)
    sharpe_ratio = (excess.mean() / vol * np.sqrt(252.0)) if vol > 1e-12 else 0.0

    print("\n--- Simulation Results ---")
    print(f"Initial Capital: ${initial_capital:,.2f}")
    print(f"Final Capital:   ${final_value:,.2f}")
    print(f"Total Return on Investment (ROI): {total_roi:.2%}")
    print(f"Annualized Sharpe Ratio: {sharpe_ratio:.2f}")
    print("--------------------------")


In [31]:
companies = pd.read_parquet('stocknet-dataset/stock_table.parquet')
tweets = pd.read_parquet('stocknet-dataset/stock_tweets_withsentiment_withemotion_withstance_nomerge.parquet')
stocks = pd.read_parquet('stocknet-dataset/stock_prices.parquet')

companies = companies.rename(columns={'symbol': 'ticker'})

companies.columns = [x.lower() for x in companies.columns]
tweets.columns = [x.lower() for x in tweets.columns]
stocks.columns = [x.lower() for x in stocks.columns]

tweets['stance_positive'] = (tweets['stance_label'] == 'Positive').astype(int)
tweets['stance_negative'] = (tweets['stance_label'] == 'Negative').astype(int)

tweets_merged = tweets.groupby(['date', 'ticker'], as_index=False).agg({
    'text': lambda x: ' '.join(x),
    'sentiment': lambda x: x.mean(),
    'emotion_anger': 'sum',
    'emotion_disgust': 'sum',
    'emotion_fear': 'sum',
    'emotion_joy': 'sum',
    'emotion_neutral': 'sum',
    'emotion_sadness': 'sum',
    'emotion_surprize': 'sum',
    'stance_positive': 'sum',
    'stance_negative': 'sum'
})




tweets_merged['date'] = pd.to_datetime(tweets_merged['date'])
stocks['date'] = pd.to_datetime(stocks['date'])

"""
master_df = stocks.merge(
    tweets_merged,
    on=["date", "ticker"]
)
"""


master_df = pd.merge(
    stocks,
    tweets_merged,
    on=["date", "ticker"],
    how='left'
)

tweet_feature_cols = ['sentiment', 'emotion_anger', 'emotion_disgust', 'emotion_fear', 'emotion_joy', 'emotion_neutral', 'emotion_sadness', 'emotion_surprize', 'stance_positive', 'stance_negative']
for col in tweet_feature_cols:
    if col in master_df.columns:
        master_df[col].fillna(0, inplace=True)



companies = companies.rename(columns={'symbol': 'ticker'})

master_df = pd.merge(master_df, companies[['ticker', 'sector', 'company']], on='ticker', how='left')


feature_cols = ['open','high','low','volume']

master_df = master_df.rename(columns={'close': 'close_price', 'company': 'company_name'})


print(f"Shape of master_df before dropping NaNs: {master_df.shape}")
#master_df.dropna(inplace=True)
print(f"Shape of master_df after dropping NaNs: {master_df.shape}")

master_df.rename(columns={'close_price': 'close'}, inplace=True)





master_df.sort_values(by=['ticker', 'date'], inplace=True)


def apply_ta_indicators(df_group):
    df_group.set_index(pd.DatetimeIndex(df_group['date']), inplace=True)
    #Trend
    df_group.ta.ema(length=12, append=True)
    df_group.ta.ema(length=26, append=True)
    df_group.ta.ema(length=50, append=True)

    df_group.ta.macd(fast=12, slow=26, signal=9, append=True)



    df_group.ta.rsi(length=14, append=True)
    df_group.ta.stochrsi(length=14, append=True)


    df_group.ta.atr(length=14, append=True)

    bb = ta.bbands(df_group['close'], length=20, std=2)
    df_group['BB_upper'] = bb['BBU_20_2.0']
    df_group['BB_middle'] = bb['BBM_20_2.0']
    df_group['BB_lower'] = bb['BBL_20_2.0']


    df_group.ta.obv(append=True)
    return df_group.reset_index(drop=True)

master_df = master_df.groupby('ticker').apply(apply_ta_indicators)


Shape of master_df before dropping NaNs: (108592, 21)
Shape of master_df after dropping NaNs: (108592, 21)


In [32]:
master_df.drop(columns=['text','adj close','sentiment','emotion_anger','emotion_disgust','emotion_fear','emotion_joy','emotion_neutral','emotion_sadness','emotion_surprize'], inplace=True)


In [33]:
master_df


date       open       high        low      close  \
ticker                                                               
AAPL   0    2012-09-04  95.108574  96.448570  94.928574  96.424286   
       1    2012-09-05  96.510002  96.621429  95.657143  95.747147   
       2    2012-09-06  96.167145  96.898575  95.828575  96.610001   
       3    2012-09-07  96.864288  97.497147  96.538574  97.205711   
       4    2012-09-10  97.207146  97.612854  94.585716  94.677139   
...                ...        ...        ...        ...        ...   
XOM    1253 2017-08-28  76.900002  76.940002  76.260002  76.470001   
       1254 2017-08-29  76.209999  76.489998  76.080002  76.449997   
       1255 2017-08-30  76.239998  76.449997  76.059998  76.099998   
       1256 2017-08-31  76.269997  76.489998  76.050003  76.330002   
       1257 2017-09-01  76.370003  76.849998  76.320000  76.570000   

                  volume ticker  stance_positive  stance_negative  \
ticker                                                              
AAPL   0      91973000.0   AAPL              0.0              0.0   
       1      84093800.0   AAPL              0.0              0.0   
       2      97799100.0   AAPL              0.0              0.0   
       3      82416600.0   AAPL              0.0              0.0   
       4     121999500.0   AAPL              0.0              0.0   
...                  ...    ...              ...              ...   
XOM    1253    8229700.0    XOM              0.0              0.0   
       1254    7060400.0    XOM              0.0              0.0   
       1255    8218000.0    XOM              0.0              0.0   
       1256   15641700.0    XOM              0.0              0.0   
       1257    7340800.0    XOM              0.0              0.0   

                       sector  ... MACDh_12_26_9  MACDs_12_26_9     RSI_14  \
ticker                         ...                                           
AAPL   0       Consumer Goods  ...           NaN            NaN        NaN   
       1       Consumer Goods  ...           NaN            NaN        NaN   
       2       Consumer Goods  ...           NaN            NaN        NaN   
       3       Consumer Goods  ...           NaN            NaN        NaN   
       4       Consumer Goods  ...           NaN            NaN        NaN   
...                       ...  ...           ...            ...        ...   
XOM    1253  Basic Matierials  ...     -0.107548      -0.972858  31.975492   
       1254  Basic Matierials  ...     -0.069077      -0.990127  31.851847   
       1255  Basic Matierials  ...     -0.054652      -1.003790  29.688704   
       1256  Basic Matierials  ...     -0.018917      -1.008519  32.913052   
       1257  Basic Matierials  ...      0.028984      -1.001273  36.200731   

             STOCHRSIk_14_14_3_3  STOCHRSId_14_14_3_3   ATRr_14   BB_upper  \
ticker                                                                       
AAPL   0                     NaN                  NaN       NaN        NaN   
       1                     NaN                  NaN       NaN        NaN   
       2                     NaN                  NaN       NaN        NaN   
       3                     NaN                  NaN       NaN        NaN   
       4                     NaN                  NaN       NaN        NaN   
...                          ...                  ...       ...        ...   
XOM    1253            35.117121            31.775404  0.786087  81.525829   
       1254            48.597552            38.712818  0.759224  81.303475   
       1255            55.025431            46.246701  0.732850  80.964170   
       1256            73.940933            59.187972  0.711932  80.569554   
       1257            85.986580            71.650981  0.698937  80.167620   

             BB_middle   BB_lower          OBV  
ticker                                          
AAPL   0           NaN        NaN   91973000.0  
       1           NaN        Na

In [34]:
columns_to_check = ['EMA_12', 'EMA_26','EMA_50','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','RSI_14','ATRr_14','STOCHRSIk_14_14_3_3','STOCHRSId_14_14_3_3','ATRr_14','BB_upper','BB_middle','BB_lower','OBV']
master_df = master_df.dropna(subset=columns_to_check)

In [35]:
feature_cols = ['open','high','low','volume',
                'stance_positive','stance_negative'
                ]

new_indicator_columns = [
    'EMA_12', 'EMA_26', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9',
    'RSI_14', 'ATRr_14', 'STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3',
    'BB_upper', 'BB_middle', 'BB_lower', 'OBV'
]
feature_cols.extend(new_indicator_columns)
print(f"Final feature columns: {feature_cols}")

Final feature columns: ['open', 'high', 'low', 'volume', 'stance_positive', 'stance_negative', 'EMA_12', 'EMA_26', 'EMA_50', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_14', 'ATRr_14', 'STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3', 'BB_upper', 'BB_middle', 'BB_lower', 'OBV']


In [36]:
master_df = master_df.rename(columns={'close': 'close_price', 'company': 'company_name'})


In [37]:
master_df.reset_index(drop=True, inplace=True)
master_df

,date,open,high,low,close_price,volume,ticker,stance_positive,stance_negative,sector,...,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,ATRr_14,BB_upper,BB_middle,BB_lower,OBV
0,2012-11-14,77.928574,78.207146,76.597145,76.697144,119292600.0,AAPL,0.0,0.0,Consumer Goods,...,-0.538077,-3.703588,25.771012,21.054629,19.582354,2.377852,94.648550,84.401357,74.154164,-1.014356e+09
1,2012-11-15,76.790001,77.071426,74.660004,75.088570,197477700.0,AAPL,0.0,0.0,Consumer Goods,...,-0.537725,-3.838019,23.573491,15.792949,19.993462,2.380310,93.761634,83.514428,73.267223,-1.211834e+09
2,2012-11-16,75.028572,75.714287,72.250000,75.382858,316723400.0,AAPL,0.0,0.0,Consumer Goods,...,-0.455544,-3.951905,24.836267,12.243346,16.363641,2.459547,92.716200,82.679214,72.642228,-8.951103e+08
3,2012-11-19,77.244286,81.071426,77.125717,80.818573,205829400.0,AAPL,0.0,0.0,Consumer Goods,...,0.002769,-3.951213,43.429047,39.692073,22.576123,2.695187,91.617665,82.201285,72.784906,-6.892809e+08
4,2012-11-20,81.701431,81.707146,79.225716,80.129997,160688500.0,AAPL,0.0,0.0,Consumer Goods,...,0.281964,-3.880722,42.011353,69.373201,40.436207,2.679612,91.027780,81.851785,72.675790,-8.499694e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104215,2017-08-28,76.900002,76.940002,76.260002,76.470001,8229700.0,XOM,0.0,0.0,Basic Matierials,...,-0.107548,-0.972858,31.975492,35.117121,31.775404,0.786087,81.525829,78.243500,74.961171,-2.688251e+08
104216,2017-08-29,76.209999,76.489998,76.080002,76.449997,7060400.0,XOM,0.0,0.0,Basic Matierials,...,-0.069077,-0.990127,31.851847,48.597552,38.712818,0.759224,81.303475,78.057500,74.811525,-2.758855e+08
104217,2017-08-30,76.239998,76.449997,76.059998,76.099998,8218000.0,XOM,0.0,0.0,Basic Matierials,...,-0.054652,-1.003790,29.688704,55.025431,46.246701,0.732850,80.964170,77.832500,74.700830,-2.841035e+08
104218,2017-08-31,76.269997,76.489998,76.050003,76.330002,15641700.0,XOM,0.0,0.0,Basic Matierials,...,-0.018917,-1.008519,32.913052,73.940933,59.187972,0.711932,80.569554,77.624500,74.679446,-2.684618e+08


In [40]:

master_df_with_target = create_target_variable(master_df)
master_df_with_target = add_horizon_targets(master_df_with_target, H=1)
payout_map = calculate_historical_payouts(master_df_with_target)

initial_capital = 100_000.0
simulation_results = run_simulation(
    master_df_with_target,
    payout_map,
    feature_cols,
    initial_capital,
    initial_training_days=INITIAL_TRAINING_DAYS,
    H=5,
    allow_overlap=True
)

display(simulation_results.head(100))
calculate_final_results(simulation_results, initial_capital)

Creating target variable...
Target variable created.

Starting simulation with 1100 initial training days... (H=5)
Adding horizon targets to master_df...
Starting predictions from day 1100 (after 1100 training days)


Simulating Trading Days:   0%|          | 0/108 [00:00<?, ?it/s]

Retraining model for ticker: AAPL
[TRAIN] AAPL: preparing data (H=5) with 1100 rows
[TRAIN] AAPL: built 1084 sequences (seq_len=12)
[TRAIN] AAPL: scaler fitted on 867 training sequences
[CAL] AAPL: taus=[0.7  0.58 0.54 0.52 0.42] | temperature=2.093
[SAVE] AAPL: stored taus
[SAVE] AAPL: stored temperature scale 2.093
[SAVE] AAPL: artifacts persisted (H=5)
Retraining model for ticker: ABB
[TRAIN] ABB: preparing data (H=5) with 1100 rows
[TRAIN] ABB: built 1084 sequences (seq_len=12)
[TRAIN] ABB: scaler fitted on 867 training sequences
[CAL] ABB: taus=[0.56 0.5  0.44 0.52 0.4 ] | temperature=0.600
[SAVE] ABB: stored taus
[SAVE] ABB: stored temperature scale 0.600
[SAVE] ABB: artifacts persisted (H=5)
Retraining model for ticker: ABBV
[TRAIN] ABBV: preparing data (H=5) with 1019 rows
[TRAIN] ABBV: built 1003 sequences (seq_len=12)
[TRAIN] ABBV: scaler fitted on 802 training sequences
[CAL] ABBV: taus=[0.5  0.46 0.4  0.36 0.48] | temperature=1.957
[SAVE] ABBV: stored taus
[SAVE] ABBV: stor

Simulating Trading Days:   1%|          | 1/108 [13:10<23:29:21, 790.29s/it]

[CAL] XOM: taus=[0.64 0.8  0.52 0.54 0.54] | temperature=1.550
[SAVE] XOM: stored taus
[SAVE] XOM: stored temperature scale 1.550
[SAVE] XOM: artifacts persisted (H=5)
[SIM] 2017-03-30 00:00:00: built 88 predictions
[PORT] Received 88 predictions for sizing.
[PORT] 56 tickers cleared prob_up ≥ 0.300 and positive expected return.
[PORT] Sector Basic Matierials: selecting PTR (p_up=0.673, kelly=0.358) with $3577.56 allocation
[PORT] Sector Conglomerates: selecting REX (p_up=0.620, kelly=0.290) with $2899.61 allocation
[PORT] Sector Consumer Goods: selecting UN (p_up=0.966, kelly=0.933) with $9330.64 allocation
[PORT] Sector Financial: selecting BAC (p_up=0.959, kelly=0.923) with $9225.13 allocation
[PORT] Sector Healthcare: selecting CELG (p_up=0.569, kelly=0.158) with $1575.33 allocation
[PORT] Sector Industrial Goods: selecting MMM (p_up=0.944, kelly=0.887) with $8865.35 allocation
[PORT] Sector Services: selecting PCLN (p_up=0.774, kelly=0.565) with $5650.88 allocation
[PORT] Sector T

...

[SIM] 2017-08-30 00:00:00: built 88 predictions
[PORT] Received 88 predictions for sizing.
[PORT] 54 tickers cleared prob_up ≥ 0.300 and positive expected return.
[PORT] Sector Basic Matierials: selecting BBL (p_up=0.995, kelly=0.990) with $9376.41 allocation
[PORT] Sector Conglomerates: selecting IEP (p_up=0.656, kelly=0.327) with $3100.79 allocation
[PORT] Sector Consumer Goods: selecting MO (p_up=0.744, kelly=0.486) with $4605.43 allocation
[PORT] Sector Financial: selecting MA (p_up=0.867, kelly=0.734) with $6954.33 allocation
[PORT] Sector Healthcare: Kelly <= 0 for CELG (p_up=0.454, b=1.050).
[PORT] Sector Industrial Goods: selecting MMM (p_up=0.936, kelly=0.871) with $8248.28 allocation
[PORT] Sector Services: selecting BABA (p_up=0.941, kelly=0.888) with $8414.53 allocation
[PORT] Sector Technology: selecting FB (p_up=0.495, kelly=0.045) with $421.96 allocation
[PORT] Sector Utilities: selecting NGG (p_up=0.764, kelly=0.521) with $4935.55 allocation
[SIM] 2017-08-30 00:00:00: 8

Simulating Trading Days: 100%|██████████| 108/108 [15:44<00:00,  8.74s/it]

[SIM] 2017-08-31 00:00:00: built 88 predictions
[PORT] Received 88 predictions for sizing.
[PORT] 52 tickers cleared prob_up ≥ 0.300 and positive expected return.
[PORT] Sector Basic Matierials: selecting BBL (p_up=0.991, kelly=0.982) with $10426.10 allocation
[PORT] Sector Conglomerates: selecting IEP (p_up=0.656, kelly=0.327) with $3476.01 allocation
[PORT] Sector Consumer Goods: selecting MO (p_up=0.743, kelly=0.483) with $5133.32 allocation
[PORT] Sector Financial: selecting MA (p_up=0.869, kelly=0.738) with $7840.33 allocation
[PORT] Sector Healthcare: Kelly <= 0 for CELG (p_up=0.429, b=1.050).
[PORT] Sector Industrial Goods: selecting MMM (p_up=0.938, kelly=0.875) with $9290.42 allocation
[PORT] Sector Services: selecting BABA (p_up=0.932, kelly=0.870) with $9241.77 allocation
[PORT] Sector Technology: selecting FB (p_up=0.495, kelly=0.044) with $465.90 allocation
[PORT] Sector Utilities: selecting NGG (p_up=0.798, kelly=0.590) with $6262.20 allocation
[SIM] 2017-08-31 00:00:00: 

,date,capital_start,capital_end,equity_start,equity_end,daily_pnl_realized,daily_return,investments_made
0,2017-03-30,100000.000000,70000.000000,100000.000000,100000.000000,0.000000,0.000000,"[{'ticker': 'UN', 'sector': 'Consumer Goods', ..."
1,2017-03-31,70000.000000,40148.860820,100000.000000,100000.000000,0.000000,0.000000,"[{'ticker': 'BAC', 'sector': 'Financial', 'pro..."
2,2017-04-03,40148.860820,30000.000000,100000.000000,100000.000000,0.000000,0.000000,"[{'ticker': 'BAC', 'sector': 'Financial', 'pro..."
3,2017-04-04,30000.000000,30000.000000,100000.000000,100000.000000,0.000000,0.000000,[]
4,2017-04-05,30000.000000,30000.000000,100000.000000,100000.000000,0.000000,0.000000,[]
...,...,...,...,...,...,...,...,...
95,2017-08-15,62125.194670,33118.026391,104574.457999,104574.457999,-217.299133,-0.002074,"[{'ticker': 'BBL', 'sector': 'Basic Matierials..."
96,2017-08-16,55881.220954,31405.496879,104684.989598,104684.989598,110.531599,0.001057,"[{'ticker': 'BBL', 'sector': 'Basic Matierials..."
97,2017-08-17,42824.543654,31474.736924,104915.789746,104915.789746,230.800148,0.002205,"[{'ticker': 'PG', 'sector': 'Consumer Goods', ..."
98,2017-08-18,34998.889930,31452.356172,104841.187239,104841.187239,-74.602506,-0.000711,"[{'ticker': 'MA', 'sector': 'Financial', 'prob..."



--- Simulation Results ---
Initial Capital: $100,000.00
Final Capital:   $106,180.23
Total Return on Investment (ROI): 6.18%
Annualized Sharpe Ratio: 4.53
--------------------------


In [41]:
folder_path = 'trained_models/'
if os.path.exists(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
else:
    print(f"Folder not found: {folder_path}")

print(f"Contents of {folder_path} after deletion attempt:")
if os.path.exists(folder_path):
    print(os.listdir(folder_path))
else:
    print("Folder does not exist.")

Contents of trained_models/ after deletion attempt:
[]
